参考： 
https://zhuanlan.zhihu.com/p/431275459

# 语音降噪概述
语音去噪(noise reduction)又称语音增强(speech enhancement)，主要是针对于有人声的音频进行处理，去除那些背景噪声，增强音频中人声的可懂性(intelligibility)。

噪声通常被分为两大类，stationary和non-stationary：

- stationary noise是指不随着时间发生变化变化的噪声，比如菜场的嘈杂声，电台的杂讯声等等。
- non-stationary noise是指随时间发生变化的噪声，比如说话时背后突然经过一辆汽车，又比如突然响起的警报声等等。

## 频减法

- 频减法： 估计出噪声，并在频域里将其幅值剪掉，再还原。原始语音$y(n)=x(n) + e(n)$, y(n)为原始语音，e(n)为噪音，x(n)是去噪后的音频，需要去求出来。
  - 截取头部作为噪音： $e(m)=y(n)[:m]$, e为噪音，y为原始语音，m和n表示采样。默认就是取前面一段作为噪声。
  - 分别计算原始音频和噪声的STFT(短时傅里叶变换),$Y(\omega)$和$E(\omega)$。
  - 根据噪声的频谱幅值，对原始音频的频谱幅值进行谱减。现在使用过减法，就是宁可错杀一千不能放过一个的做法，通过控制过减因子来控制残留的噪音。
  - 对$\vert \hat X(\omega) \vert$做平滑处理，使得声音失真没那么严重，使用scipy.signal.fftconvolve来实现这一过程。
  - 结合原始音频的相位，还原谱减后的音频。这就是个反向STFT的过程

$$
\vert \hat X(\omega) \vert = \begin{cases}
\vert Y(\omega) \vert - \alpha \vert E(\omega) \vert_{mean}, &if \quad \vert Y(\omega) \vert \gt (\alpha + \beta)\vert E(\omega)\vert_{mean} \\
\beta\vert E(\omega)\vert, &otherwise \\
\end{cases} \tag{过减法}
$$

$\alpha \in [0,+\infty)$是过减因子, $\beta \in [0,1]$是谱下限幅度

## 维纳滤波法

维纳滤波法(wiener filter)也是一个比较经典的传统做法，它的本质是估计出一个线性滤波器，也就是一个向量，这个滤波器会对不同的频段进行不同程度的抑制，其保真效果会比谱减法要好一些




## 深度语音去噪

## demucs
facebook出品的机遇DEMUCS的denoiser。DEMUCS之前是用于音频分轨(source separation)的，去噪的本质其实也就是把人声轨给分离出来，与其说是去噪，不如说是提取人声更为合理一些。当然，这个都是由数据控制的。其目的是用神经网络构建一个函数f ff使得式( 2 − 1 ) (2-1)(2−1)中的

x ( n ) = f ( y ( n ) ) (3-1) x(n) = f(y(n)) \tag{3-1}
x(n)=f(y(n))(3-1)

denoiser的模型架构非常简明易懂，也非常轻量，可以用于实时的语音去噪，其结构示意图如下图3-1所示

## FRCRN语音降噪

在ModelScope上开放的FRCRN语音降噪-16K模型，是基于频率循环 CRN (FRCRN) 新框架开发出来的。该框架是在卷积编-解码(Convolutional Encoder-Decoder)架构的基础上，通过进一步增加循环层获得的卷积循环编-解码(Convolutional Recurrent Encoder-Decoder)新型架构，可以明显改善卷积核的视野局限性，提升降噪模型对频率维度的特征表达，尤其是在频率长距离相关性表达上获得提升，可以在消除噪声的同时，对语音进行更针对性的辨识和保护。
另外，我们引入前馈序列记忆网络（Feedforward Sequential Memory Network: FSMN）来降低循环网络的复杂性，以及结合复数域网络运算，实现全复数深度网络模型算法，不仅更有效地对长序列语音进行建模，同时对语音的幅度和相位进行同时增强。相关模型在IEEE/INTERSpeech DNS Challenge上有较好的表现。该模型神经网络结构如下图所示。

https://modelscope.cn/models/damo/speech_frcrn_ans_cirm_16k/summary

- RNNoise: [A Hybrid DSP/Deep Learning Approach to Real-Time Full-Band Speech Enhancement](), [开源代码](https://github.com/xiph/rnnoise/), [效果地址](https://jmvalin.ca/demo/rnnoise/)

[学习地址](https://zhuanlan.zhihu.com/p/397288851)

信号模型：
- 窗口20ms、出口之间的overlap 10ms，窗函数 $w(n)=\sin[\pi/2 \sin ^2(\pi n /N)]$, N为窗长。


## Wav-U-net

论文地址： [WAVE-U-NET: A MULTI-SCALE NEURAL NETWORK FOR END-TO-END AUDIO SOURCE SEPARATION]()

Wave-U-Net 是一种卷积神经网络，适用于音频源分离任务，它直接作用于原始音频波形（wav）。 它是 U-Net 架构对一维时域信号的一种变形， 实现端到端音频源分离。通过一系列下采样和上采样块（包括 1D 卷积和下/上采样过程），在多个尺度/抽象级别和时间分辨率上计算特征。

[开源代码](https://github.com/f90/Wave-U-Net)
[效果体验](https://sisec18.unmix.app/#/unmix/Side%20Effects%20Project%20-%20Sing%20With%20Me/STL1)

## Low-Complexity Acoustic Echo Cancellation with Neural Kalman Filtering

"Low-Complexity Acoustic Echo Cancellation with Neural Kalman Filtering" [arXiv](https://arxiv.org/abs/2207.11388). (Accepted by ICASSP 2023)

[源码](https://github.com/fjiang9/NKF-AEC)